### Read CSV 

In [7]:
import pandas as pd

R = .10 # filter ratio 
my_file = "MDACC_compiledrpkm_101genes.csv"
df = pd.read_csv(my_file)
genes = ['UBD','CXCL13','CXCL11','IDO1','IGHM','CCL5','CXCL10','NTN3','GZMB','IGKC','LOC652493','PSMB9','GBP1','FGL2','IL23A','CD52','RARRES3','IGJ','RTP4','TNFSF10','ITM2A','TNFAIP8','GABRP','KYNU','S100A8','PTGDS','PMAIP1','FABP7','KMO','XBP1','CHI3L1','DEFB1','DUSP5','ART3','NEK2','C7','TSC22D3','CDC20','SPTLC2','MYBL1','IL33','SIDT1','LBP','GALNT7','JAM2','S100A7','PI3','GCHFR','PLEKHB1','MFAP4','TRIM68','TCF7L1','INPP4B','FOXA1','SPRR1B','FGFR4','ADRA2A','SFRP1','ABCA8','FASN','CRAT','KCNK5','ALDH3B2','MID1','CYP4F8','DHCR24','IGFBP4','HGD','LHFP','BLVRB','SPDEF','CROT','GPR87','ZCCHC24','THBS4','SEMA3C','OGN','KRT6A','COL5A2','DBI','AKAP12','HTRA1','FOXC1','CD36','MIA','S100A1','TFAP2B','KIAA1324','SERHL2','COL5A1','UGT2B28','KRT16','KRT6B','SRPX','SOX10','AZGP1','APOD','KRT17','KRT14','COL2A1','ASPN','SCRG1']
n = int(R*len(genes))
#df.head()

### Filter by genes not contained in genes list

In [8]:
samples = df.columns
tdf = df[df['gene_short_name'].isin(genes)]
#tdf.head()

### Filter bad samples

This creates a list of lists correlating each sample to the genes in which a value of 0 was determined

In [9]:
shit_list = []
bad_samples = []
good_samples = df.columns[1:].tolist()
for sample in tdf.columns[1:]:
    tmp_list = tdf.loc[tdf[sample] == 0].gene_short_name.tolist()
    tmp_list.append(sample)
    if len(tmp_list) - 1 > n:
        shit_list.append(tmp_list)
        bad_samples.append(sample)
        good_samples.remove(sample)
        
## main dataframe drop        
gdf = df.drop(bad_samples, axis=1)
bdf = df.drop(good_samples, axis=1)
#bad_df.head(20)
#gdf.to_csv('out_file.csv')
#bdf.to_csv('out_file.csv')

#### Formatting & QC export

In [11]:
## preparing QC file
shit_list.sort(key=len)
shit_list.reverse()
d = {}
for sample in shit_list:
    sample_name = sample[-1:][0]
    d[sample_name] = sample[:-1]
bad_df = pd.DataFrame.from_dict(d, orient='index').T

## formatting magic that gets rid of the 'None's'
pm = bad_df.apply(sorted,key=pd.isnull)
bad_df = pm[~pd.isnull(pm).all(1)].fillna('')
bad_df.head(20)

bad_df.to_csv('MDACC_compiled_101zeroes.csv')